In [1]:
import os.path as osp
import cv2
import pandas as pd
import re

In [6]:
def images_to_video(image_paths, output_video_path, fps=30):
    # Read the first image to get the dimensions
    frame = cv2.imread(image_paths[0])
    height, width, layers = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for image_path in image_paths:
        video.write(cv2.imread(image_path))

    # Release the video writer object
    video.release()

## Ver1 Video Sequence

In [2]:
# Load the image lists to generate videos
image_list_record_root = '/prj/qct/mmrd-cv/wonderland_data/3DFR_Data_wrap4d/Pair_man/PAC/meta/train'
image_root = '/prj/qct/mmrd-cv/wonderland_data/3DFR_Data/3dMD/PAC'
out_root = '/prj/qct/mmrd-cv/wonderland_data/3DFR_Data/3dMD/PAC/live-portrait-data/processed_videos'
image_lists_records_out_path = '/prj/qct/mmrd-cv/wonderland_data/3DFR_Data/3dMD/PAC/live-portrait-data/image_lists_records.pkl'

# subject_names = ['atalie', 'keli', 'matt', 'ning']
image_lists_record_files = [
    'data_parsed_List_atalie_man_process_list_train.csv',
    'data_parsed_List_keli_man_process_list_train.csv',
    'data_parsed_List_matt_man_process_list_train.csv',
    'data_parsed_List_ning_man_process_list_train.csv'
]
# get the subject names using regex
subject_names = [re.search(r'data_parsed_List_(.*)_man_process_list_train.csv', f).group(1) for f in image_lists_record_files]
print(subject_names)

image_lists_record_files = [osp.join(image_list_record_root, p) for p in image_lists_record_files]
image_lists_records = []

def add_unified_suffix(path):
    root, name = osp.split(path)
    stem, ext = osp.splitext(name)
    if not stem.endswith('05_C'):
        stem += '.05_C'
    return osp.join(image_root, root, stem+ext)

for f in image_lists_record_files:
    img_list = pd.read_csv(f)['tar_bg'].tolist()
    img_list = [add_unified_suffix(p) for p in img_list]
    image_lists_records.append(img_list)

['atalie', 'keli', 'matt', 'ning']


# Ver2 Video Sequence

In [10]:
# Load the image lists to generate videos
image_list_record_root = '/mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/meta'
image_root = '/mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose'
out_root = '/mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/processed_videos_ver2'
image_lists_records_out_path = '/mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/image_lists_records.pkl'

subject_names = ['atalie', 'keli', 'matt', 'ning']
image_lists_record_files = [
    'data_parsed_List_subject5_cap_man_process_list_train.csv',
    'data_parsed_List_subject5_tied_hair_man_process_list_train.csv',
    'data_parsed_List_subject25_cap_man_process_list_train.csv',
    'data_parsed_List_subject25_tied_hair_man_process_list_train.csv'
]
# get the subject names using regex
subject_names = [re.search(r'data_parsed_List_(.*)_man_process_list_train.csv', f).group(1) for f in image_lists_record_files]
print(subject_names)

image_lists_record_files = [osp.join(image_list_record_root, p) for p in image_lists_record_files]
image_lists_records = []

def add_unified_suffix(path):
    return osp.join(image_root, path+'.syn.png')

for f in image_lists_record_files:
    img_list = pd.read_csv(f)['tar'].tolist()
    img_list = [add_unified_suffix(p) for p in img_list]
    image_lists_records.append(img_list)

['subject5_cap', 'subject5_tied_hair', 'subject25_cap', 'subject25_tied_hair']


# Processing

In [11]:
# Check file existance status
for il in image_lists_records:
    print(osp.exists(il[0]), osp.exists(il[-1]))

True True
True True
True True
True True


In [12]:
# Generate Videos
for idx, img_list in enumerate(image_lists_records):
    subj_name = subject_names[idx]
    out_video_path = osp.join(out_root, f'{subj_name}.mp4')
    print("Now processing: ", out_video_path)
    
    # Generate video from images
    images_to_video(img_list, out_video_path, fps=30)


Now processing:  /mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/processed_videos_ver2/subject5_cap.mp4
Now processing:  /mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/processed_videos_ver2/subject5_tied_hair.mp4
Now processing:  /mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/processed_videos_ver2/subject25_cap.mp4
Now processing:  /mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/processed_videos_ver2/subject25_tied_hair.mp4


In [30]:
# Save filenames
import pickle as pkl
image_lists_records_out_path = '/prj/qct/mmrd-cv/wonderland_data/3DFR_Data/3dMD/PAC/live-portrait-data/image_lists_records.pkl'
with open(image_lists_records_out_path, 'wb') as f:
    pkl.dump(image_lists_records, f)